# Data pre-processing notebook
### Dropping non-songs, encoding categorical variables

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../../tracks.csv")
df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


Data cleanup

In [2]:
!pip install category_encoders

In [3]:
import category_encoders as ce

def clean_data(df):
    # Drop the id column, name, release_date
    ### Engineer year feature
    x = df.copy()
    drop_cols = ['id', 'name', 'release_date', 'id_artists', 'artists']
    x = x.drop(labels=drop_cols, axis=1)
    
    # Drop zero tempo songs
    x = x[x['tempo']!=0]
    
    # Do one-hot encoding for key
    ohe = ce.one_hot.OneHotEncoder(cols=['key'])
    x = ohe.fit_transform(x)
    
    # Do ordinal encoding for artists
    #oe = ce.ordinal.OrdinalEncoder()
    #x_oe = oe.fit_transform(x[['artists']])
    #x['artists'] = x_oe
    
    # Normalize numeric features
    num_features = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature']
    
    for i in num_features:
        col_mean = x[i].values.mean()
        col_var = np.var(df[i])
        col_norm = [(o - col_mean) / col_var**(1/2) for o in x[i].values]
        x[i] = col_norm
    
    return x


In [4]:

clean_df = clean_data(df)
print(f'Old df shape: {df.shape}\nOld df columns: {[i for i in df.columns]}\n\n'
        f'Cleaned df shape: {clean_df.shape}\nClean df columns: {[i for i in clean_df.columns]}')

/Users/Carl/miniconda3/envs/U4-S2-NN/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Old df shape: (586672, 20)
Old df columns: ['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists', 'id_artists', 'release_date', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

Cleaned df shape: (586344, 26)
Clean df columns: ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_12', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']


In [5]:
clean_df.describe()

,popularity,duration_ms,explicit,danceability,energy,key_1,key_2,key_3,key_4,key_5,...,key_12,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,586344.000000,586344.000000,586344.000000,586344.000000,586344.000000,...,586344.000000,5.863440e+05,586344.000000,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05
mean,2.908365e-17,6.126956e-17,-6.980077e-18,4.907382e-16,-1.126507e-16,0.127717,0.071086,0.125781,0.036711,0.091388,...,0.111044,-7.367859e-18,0.658792,-1.192430e-16,1.983505e-16,2.088206e-16,6.223902e-17,5.021777e-17,5.506505e-17,-1.275803e-16
std,9.998227e-01,9.952174e-01,1.000212e+00,9.970543e-01,9.994280e-01,0.333775,0.256969,0.331603,0.188050,0.288161,...,0.314187,9.963881e-01,0.474115,1.000185e+00,9.998803e-01,9.992351e-01,9.996187e-01,9.989943e-01,9.958386e-01,9.813526e-01
min,-1.500701e+00,-1.697285e+00,-2.148490e-01,-3.074662e+00,-2.152405e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.802915e+00,0.000000,-5.832490e-01,-1.289455e+00,-4.244867e-01,-1.129392e+00,-2.144603e+00,-2.957428e+00,-8.190745e+00
25%,-7.930492e-01,-4.340974e-01,-2.148490e-01,-6.677147e-01,-7.869069e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-5.279452e-01,0.000000,-3.942474e-01,-1.011674e+00,-4.244867e-01,-6.271283e-01,-8.018040e-01,-7.687832e-01,2.630195e-01
50%,-3.096308e-02,-1.195375e-01,-2.148490e-01,7.881222e-02,2.683466e-02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.884565e-01,1.000000,-3.369910e-01,-7.971846e-02,-4.243953e-01,-4.063232e-01,4.423698e-02,-3.746548e-02,2.630195e-01
75%,7.311231e-01,2.673159e-01,-2.148490e-01,7.350335e-01,8.167594e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7.307683e-01,1.000000,-1.585512e-01,9.580173e-01,-3.888886e-01,3.477778e-01,8.398260e-01,5.982336e-01,2.630195e-01
max,3.942772e+00,4.260912e+01,4.656401e+00,2.571249e+00,1.817066e+00,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,3.060351e+00,1.000000,4.814415e+00,1.565752e+00,3.322689e+00,4.264763e+00,1.736319e+00,4.295441e+00,2.376461e+00


In [8]:
clean_df.to_csv('../../songs_cleaned.csv')

FileNotFoundError: [Errno 2] No such file or directory: './Data/songs_cleaned.csv'